# EarthRanger IO

## Setup

### Ecoscope

In [ ]:
!pip install ecoscope &> /dev/null

In [ ]:
import getpass
import os
import sys

import geopandas as gpd
import pandas as pd
from erclient.client import ERClientException
from shapely.geometry import Point

import ecoscope

ecoscope.init()

### Google Drive Setup

In [ ]:
output_dir = "Ecoscope-Outputs"

if "google.colab" in sys.modules:
    from google.colab import drive

    drive.mount("/content/drive/", force_remount=True)
    output_dir = os.path.join("/content/drive/MyDrive/", output_dir)

os.makedirs(output_dir, exist_ok=True)

### Connect to EarthRanger

In [ ]:
ER_SERVER = os.getenv("ER_SERVER", "https://mep-dev.pamdas.org")
ER_USERNAME = os.getenv("ER_USERNAME", "username")  # replace with your username

ER_PASSWORD = os.getenv("ER_PASSWORD", "")

if not ER_PASSWORD:
    ER_PASSWORD = getpass.getpass("Please enter your ER password: ")

er_io = ecoscope.io.EarthRangerIO(
    server=ER_SERVER,
    username=ER_USERNAME,
    password=ER_PASSWORD,
    tcp_limit=5,
    sub_page_size=4000,
)

## GET `Event`

### By `event_ids`
Use this approach to download an individual `Event` if you know its id. Downloading from multiple ids at once is currently unsupported in ER.

In [ ]:
er_io.get_events(
    event_ids="855903a6-dfab-4b91-b75d-c993bea1208d",  # Elephant sighting event
)

### By `event_type`
Use this approach if you want to download events of a given event type (using the EventType ID from ER)

In [ ]:
er_io.get_events(
    event_type=["518bcc0a-fd6a-4607-a066-56f716720b0c"],  # MEP Elephant Sighting
    since=pd.Timestamp("2019-05-01").isoformat(),
    until=pd.Timestamp("2020-05-01").isoformat(),
)

### Unpack `event_details` column of `JSON` data into individual columns

In [ ]:
events_df = er_io.get_events(event_type=["e00ce1f6-f9f1-48af-93c9-fb89ec493b8a"])  # MEP Distance Survey Count
events_df.event_details

In [ ]:
er_io._normalize_column(events_df, "event_details")
events_df

## POST `Event`

First delete any existing events:

In [ ]:
event_df = er_io.get_events(
    event_type=["b0d66402-34cb-4e9a-9715-2ac4eeebb574"],  # Carcass
)
if not event_df.empty:
    for i, value in event_df.iterrows():
        er_io.delete_event(event_id=event_df.loc[i, "id"])

Define events to be uploaded:

In [ ]:
new_event = [
    {
        "event_details": {
            "carcassrep_ageofanimal": None,
            "carcassrep_ageofcarcass": None,
            "carcassrep_causeofdeath": None,
            "carcassrep_sex": None,
            "carcassrep_species": None,
            "carcassrep_trophystatus": None,
        },
        "event_type": "carcass_rep",
        "icon_id": "carcass_rep",
        "id": "e29f9078-ee0a-4f06-b685-92e9ff266e9b",
        "location": {"latitude": -27.12759, "longitude": -109.40804},
        "priority": 100,
        "state": "resolved",
        "time": pd.Timestamp("2022-03-15 15:44:00-0700"),
        "title": "Carcass",
    }
]

Read events as dataframe:

In [ ]:
new_event_df = pd.DataFrame.from_dict(new_event)
new_event_df

Upload:

In [ ]:
try:
    er_io.post_event(events=new_event_df.iloc[[0]])
except ERClientException as e:
    print(e)

## PATCH `Event`

Define updated values

In [ ]:
updated_event = pd.DataFrame(
    [
        {
            "event_details": {
                "carcassrep_ageofanimal": "adult",
                "carcassrep_ageofcarcass": "less_than_a_day",
                "carcassrep_causeofdeath": "unnaturalshot",
                "carcassrep_sex": "female",
                "carcassrep_species": "elephant",
                "carcassrep_trophystatus": "removed",
            }
        }
    ]
)

Get and display current values

In [ ]:
event_df = er_io.get_events(
    event_type=["b0d66402-34cb-4e9a-9715-2ac4eeebb574"],  # Carcass
)

event_df.loc[0, "event_details"]

Patch and confirm values have been updated

In [ ]:
id_to_patch = event_df.loc[0, "id"]
er_io.patch_event(event_id=id_to_patch, events=updated_event).loc[0, "event_details"]

## GET `Subjects`

### by `id`

In [ ]:
er_io.get_subjects(id="64444ed7-72ec-4531-a2b1-fb25c7197b2d")  # Habiba

### By `SubjectGroup` `group_name`

In [ ]:
er_io.get_subjects(group_name="Elephants")

## GET `Observations`

The output is an Ecoscope `Relocations` dataframe that has a core set of columns: `groupby_col`, `fixtime`, `junk_status`.

All other columns are prepended by `extra__`. The `groupby_col` will be the `subject_id` and the index of the dataframe is the observation indices.

Classic ER `Observation` format can be returned by passing in parameter `relocations=False`.

Filter observation by setting `filter` to one of: None, 0, 1, 2 or 3
- None returns everything
- 0 filters out everything but observations with exclusion flag 0 (Pass back clean data)
- 1 filters out everything but observations with exclusion flag 1 (Pass back manually filtered data)
- 2 filters out everything but observations with exclusion flag 2 (Ppass back automatically filtered data)
- 3 filters out everything but observations with exclusion flag 2 or 1 (Pass back both manual and automatic filtered data)

### By `SubjectGroup` `group_name`:

In [ ]:
since = pd.Timestamp("2008-01-01").isoformat()
until = pd.Timestamp("2020-01-01").isoformat()

In [ ]:
relocs = er_io.get_subjectgroup_observations(
    group_name="Elephants",
    filter=0,
    since=since,
    until=until,
    include_details=True,
    include_inactive=True,
    include_source_details=True,
    include_subject_details=True,
    include_subjectsource_details=False,
)
relocs

### By `subject_id`

In [ ]:
relocs = er_io.get_subject_observations(
    subject_ids=[
        "64444ed7-72ec-4531-a2b1-fb25c7197b2d",  # Habiba
        "b8be28f7-8c20-46d9-85a5-fd817351bde5",  # Salif Keita
    ],
    since=since,
    until=until,
    filter=0,
    include_details=True,
    include_subject_details=True,
    include_source_details=True,
    include_subjectsource_details=False,
)
relocs

### By `source_id`

In [ ]:
relocs = er_io.get_source_observations(
    source_ids=[
        "5e6ce0f4-5deb-47b4-a65b-539f00dc716d",  # Habiba's Source
        "ea18dc70-8bcf-4eb5-8329-a0716f309fa5",  # Salif Keita's Source
    ],
    since=since,
    until=until,
    filter=0,
    include_details=True,
    include_source_details=True,
)
relocs

## GET `Patrols`

In [ ]:
er_io.get_patrols(patrol_type=["0ef3bf48-b44c-4a4e-a145-7ab2e38c9a57"])  # MEP Distance Count Survey

## GET `Observations` for a `Patrol`

In [ ]:
patrol_df = er_io.get_patrols()

relocs = er_io.get_observations_for_patrols(
    patrol_df,
    include_source_details=True,
    include_subject_details=True,
    include_subjectsource_details=False,
)

## GET `Users`

In [ ]:
df = pd.DataFrame(er_io.get_users())
df

## POST `Observation`

First delete any existing observations

In [ ]:
elephants = er_io.get_subjectgroup_observations(group_name="Elephants", since=pd.Timestamp("2022-01-01").isoformat())

if not elephants.empty:
    for i, value in elephants.iterrows():
        er_io.delete_observation(observation_id=elephants.loc[i, "extra__id"])

Create new observations from existing sources

In [ ]:
observations = [
    {
        "fixtime": pd.Timestamp("2023-04-24 11:03:31").isoformat(),
        "geometry": Point(1.99, 2.99),
        "source_id": "5e6ce0f4-5deb-47b4-a65b-539f00dc716d",  # Habiba's source
    },
    {
        "fixtime": pd.Timestamp("2023-02-01 14:02:05").isoformat(),
        "geometry": Point(1.345, 2.3303),
        "source_id": "5e6ce0f4-5deb-47b4-a65b-539f00dc716d",  # Habiba's source
    },
    {
        "fixtime": pd.Timestamp("2023-03-21 06:03:27").isoformat(),
        "geometry": Point(3.4455, -3.4554),
        "source_id": "ea18dc70-8bcf-4eb5-8329-a0716f309fa5",  # Salif Keita's source
    },
]

gdf = gpd.GeoDataFrame.from_dict(observations)
gdf

Post observations

In [ ]:
er_io.post_observations(observations=gdf, source_id_col="source_id", recorded_at_col="fixtime")

## POST `SubjectSource`

In [ ]:
er_io.post_subjectsource(
    subject_id="b8be28f7-8c20-46d9-85a5-fd817351bde5",  # Salif Keita
    source_id="ea18dc70-8bcf-4eb5-8329-a0716f309fa5",  # Salif Keita's source
    lower_bound_assigned_range=pd.Timestamp.utcnow().isoformat(),
    upper_bound_assigned_range=(pd.Timestamp.utcnow() + pd.Timedelta(days=9000)).isoformat(),
    additional={},
)

## Export to File

### Create test data

Skip this cell if you would like to provide your own data

In [ ]:
relocs = er_io.get_subjectgroup_observations(
    group_name="Elephants",
    filter=0,
    since=since,
    until=until,
    include_details=True,
    include_inactive=True,
    include_source_details=True,
    include_subject_details=True,
    include_subjectsource_details=False,
)

### GeoPackage (.gpkg) 
(GeoPackage does not support columns with type `list` so we `drop` them.)

In [ ]:
relocs.drop(
    columns=relocs.columns[relocs.applymap(lambda x: isinstance(x, list)).any()],
    errors="ignore",
    inplace=True,
)

relocs.to_file(os.path.join(output_dir, "easter_island.gpkg"), layer="easter_island")

### CSV

In [ ]:
relocs.to_csv(os.path.join(output_dir, "observations.csv"))

### Python Pickle (.pkl)

In [ ]:
relocs.to_pickle(os.path.join(output_dir, "observations.pkl"))

### Apache Feather (.feather)

In [ ]:
relocs.to_feather(os.path.join(output_dir, "observations.feather"))